# CREATE PROCEDURE

**Applies to:** Databricks SQL, Databricks Runtime 17.0+, Unity Catalog only

Creates a stored procedure (a persisted SQL script) in Unity Catalog. Use **CALL** to invoke. Parameters: **IN** (default), **OUT**, **INOUT**. Characteristics: **LANGUAGE SQL**, **SQL SECURITY INVOKER** (required), COMMENT, etc. The procedure body is a compound statement (`BEGIN ... END`).

In [0]:
USE CATALOG healthcare_dev;
USE SCHEMA gold;

## Create procedure: claim summary by state (IN parameter, returns result set)

In [0]:
CREATE OR REPLACE PROCEDURE claim_summary_by_state(IN state_filter STRING)
  LANGUAGE SQL
  SQL SECURITY INVOKER
  COMMENT 'Returns claim count and total amount per state from gold fact + dim_cms_beneficiary'
  AS BEGIN
    IF state_filter IS NULL OR TRIM(state_filter) = '' THEN
      SELECT t.sp_state_code AS state, COUNT(*) AS claim_count, SUM(t.clm_pmt_amt) AS total_amount
      FROM (
        SELECT b.sp_state_code, f.clm_pmt_amt FROM fact_cms_outpatient_claims f JOIN dim_cms_beneficiary b ON f.beneficiary_key = b.beneficiary_key
        UNION ALL
        SELECT b.sp_state_code, f.clm_pmt_amt FROM fact_cms_inpatient_claims f JOIN dim_cms_beneficiary b ON f.beneficiary_key = b.beneficiary_key
      ) t GROUP BY t.sp_state_code;
    ELSE
      SELECT t.sp_state_code AS state, COUNT(*) AS claim_count, SUM(t.clm_pmt_amt) AS total_amount
      FROM (
        SELECT b.sp_state_code, f.clm_pmt_amt FROM fact_cms_outpatient_claims f JOIN dim_cms_beneficiary b ON f.beneficiary_key = b.beneficiary_key WHERE b.sp_state_code = state_filter
        UNION ALL
        SELECT b.sp_state_code, f.clm_pmt_amt FROM fact_cms_inpatient_claims f JOIN dim_cms_beneficiary b ON f.beneficiary_key = b.beneficiary_key WHERE b.sp_state_code = state_filter
      ) t GROUP BY t.sp_state_code;
    END IF;
  END

In [0]:
CALL claim_summary_by_state('NY');

In [0]:
CALL claim_summary_by_state(NULL);

## Create procedure with OUT parameters: total claim count and total amount

In [0]:
CREATE OR REPLACE PROCEDURE get_claim_totals(OUT total_count BIGINT, OUT total_amount DECIMAL(18,2))
  LANGUAGE SQL
  SQL SECURITY INVOKER
  AS BEGIN
    SET total_count = (SELECT COUNT(*) FROM fact_cms_outpatient_claims) + (SELECT COUNT(*) FROM fact_cms_inpatient_claims);
    SET total_amount = (SELECT COALESCE(SUM(clm_pmt_amt), 0) FROM fact_cms_outpatient_claims) + (SELECT COALESCE(SUM(clm_pmt_amt), 0) FROM fact_cms_inpatient_claims);
  END

In [0]:
BEGIN
  DECLARE cnt BIGINT;
  DECLARE amt DECIMAL(18,2);
  CALL get_claim_totals(cnt, amt);
  SELECT cnt AS total_count, amt AS total_amount;
END